In [1]:
import imageio
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
from scipy.ndimage import median_filter
import os

In [2]:
def readImg(path):
    img = imageio.imread(path)
    im = np.array(img,dtype='uint8')
    return im

def writeImg(path,buf):
    imageio.imwrite(path,buf)
    
    
def dask_median_filter(x, size):
    return x.map_overlap(median_filter, size, size=size)

In [3]:
from dask_image.ndfilters import median_filter

def main(n):
    data_dir = ''
    file = os.path.join(data_dir,'lena_noisy.jpg')
    img = readImg(file)
    #print('IMG\n',img_buf)
    nx=img.shape[0]
    ny=img.shape[1]
    img_buf = da.from_array(img, chunks=(nx//n + 1, ny//n + 1, 3))
    print('SHAPE',img_buf.shape)
    
    ###########################################################################
    #
    # SPLT IMAGES IN NB_PARTITIONS PARTS
    
    new_img_buf = dask_median_filter(img_buf, (3, 3, 3))

    
    ###########################################################################
    #
    # COMPUTE NEW IMAGE RESULTS FROM RESULT RDD
    
    #new_img_buf = np.concatenate(arrays)
    #print(f"Final shape : {new_img_buf.shape}")
    computed = new_img_buf.compute()
    
    filter_file = os.path.join(data_dir,'lena_filter_dask.jpg')
    writeImg(filter_file, computed)
    print('CREATED NEW PICTURE FILE')

In [4]:
import time

main(1)


times = []
for i in [1, 2, 4, 8]:
    start_t = time.time()
    main(i)
    times.append(time.time() - start_t)
print(times)

SHAPE (128, 128, 3)


ValueError: `dtype` inference failed in `map_blocks`.

Please specify the dtype explicitly using the `dtype` kwarg.

Original error is below:
------------------------
AttributeError("'numpy.ndarray' object has no attribute 'map_overlap'")

Traceback:
---------
  File "/home/arthurviens/local/anaconda3/envs/dask/lib/python3.9/site-packages/dask/array/core.py", line 397, in apply_infer_dtype
    o = func(*args, **kwargs)
  File "/home/arthurviens/local/anaconda3/envs/dask/lib/python3.9/site-packages/dask_image/ndfilters/_order.py", line 61, in median_filter
    result = image.map_overlap(
